In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [3]:
x_test.shape

(30, 224, 224, 3)

In [4]:
#x_train /= 255????

In [5]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
import sys
sys.path.insert(0,'..')
from utils import lossprettifier
from Classifier.ResNet import *

C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\f

In [6]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [7]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [8]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [9]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [10]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating ResNet18 model
model = ResNet18((224, 224, 3), 4) 

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)


reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...

Epoch 1/50
32/32 [==============================] - 20s 637ms/step - loss: 3.8589 - accuracy: 0.5903 - val_loss: 123.1791 - val_accuracy: 0.3333
Epoch 2/50
32/32 [==============================] - 5s 155ms/step - loss: 3.2788 - accuracy: 0.9041 - val_loss: 120.2422 - val_accuracy: 0.3333
Epoch 3/50
32/32 [==============================] - 5s 150ms/step - loss: 2.9896 - accuracy: 0.9905 - val_loss: 66.7019 - val_accuracy: 0.3333
Epoch 4/50
32/32 [==============================] - 5s 153ms/step - loss: 2.9650 - accuracy: 0.9918 - val_loss: 22.4536 - val_accuracy: 0.3333
Epoch 5/50
32/32 [==============================] - 5s 154ms/step - loss: 2.9528 - accuracy: 0.9918 - val_loss: 17.2466 - val_accuracy: 0.3667
Epoch 6/50
32/32 [==============================] - 5s 154ms/step - loss: 2.9333 - accuracy: 0.9968 - val_loss: 8.3205 - val_accuracy: 0.5000
Epoch 7/50
32/

In [11]:
modelPath = "ResNet18_COVID19.h5"
resultPath = 'ResNet18_COVID19.txt'

In [12]:
# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

y_test_oh = dense_to_one_hot(y_test,num_clases=4)

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 3.87(+0.00%) 	| LossAB: 123.18(+0.00%) 	
Epoch     1 | LossA: 3.27(-15.53%) ▼	| LossAB: 120.24(-2.38%) ▼	
Epoch     2 | LossA: 2.99(-8.62%) ▼	| LossAB: 66.70(-44.53%) ▼	
Epoch     3 | LossA: 2.96(-0.87%) ▼	| LossAB: 22.45(-66.34%) ▼	
Epoch     4 | LossA: 2.95(-0.22%) ▼	| LossAB: 17.25(-23.19%) ▼	
Epoch     5 | LossA: 2.93(-0.75%) ▼	| LossAB: 8.32(-51.76%) ▼	
Epoch     6 | LossA: 2.92(-0.22%) ▼	| LossAB: 7.10(-14.72%) ▼	
Epoch     7 | LossA: 2.93(+0.05%) ▲	| LossAB: 4.76(-32.90%) ▼	
Epoch     8 | LossA: 2.91(-0.57%) ▼	| LossAB: 5.01(+5.21%) ▲	
Epoch     9 | LossA: 2.90(-0.41%) ▼	| LossAB: 4.72(-5.81%) ▼	
Epoch    10 | LossA: 2.88(-0.51%) ▼	| LossAB: 4.50(-4.70%) ▼	
Epoch    11 | LossA: 2.88(-0.25%) ▼	| LossAB: 4.30(-4.31%) ▼	
Epoch    12 | LossA: 2.86(-0.39%) ▼	| LossAB: 4.38(+1.77%) ▲	
Epoch    13 | LossA: 2.86(-0.25%) ▼	| LossAB: 4.51(+3.01%) ▲	
Epoch    14 | LossA: 2.85(-0.30%) ▼	| LossAB: 4.46(-1.23%) ▼	
Epoch    15 | LossA: 2.84(-0.25%) ▼	| LossAB: 4.49(+0.74%)

In [13]:
y_pred = model.predict(x_test)
#y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67        10
           1       0.47      0.70      0.56        10
           2       0.43      0.30      0.35        10

    accuracy                           0.53        30
   macro avg       0.55      0.53      0.53        30
weighted avg       0.55      0.53      0.53        30

